Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f, encoding='latin1')
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (30000, 28, 28) (30000,)
Validation set (5000, 28, 28) (5000,)
Test set (5000, 28, 28) (5000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (30000, 784) (30000, 10)
Validation set (5000, 784) (5000, 10)
Test set (5000, 784) (5000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
np.where(train_labels == 1)[1]

array([1, 1, 7, ..., 7, 6, 9], dtype=int64)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression(penalty='l2')
lr.fit(train_dataset, np.where(train_labels == 1)[1])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
print(accuracy_score(lr.predict(test_dataset), np.where(test_labels == 1)[1]))
print(accuracy_score(lr.predict(valid_dataset), np.where(valid_labels == 1)[1]))

0.8818
0.803


In [9]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases = tf.Variable(tf.zeros([hidden_nodes]))    
  weights1 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases1 = tf.Variable(tf.zeros([num_labels]))   
    
  layer1_out = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  
  # Training computation.
  logits = tf.matmul(layer1_out, weights1) + biases1 + tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights1)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)


  valid_layer1_out = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_layer1_out, weights1) + biases1)
  test_layer1_out = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_layer1_out, weights1) + biases1)

In [10]:
num_steps = 5000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 353.858093
Minibatch accuracy: 13.3%
Validation accuracy: 21.9%
Minibatch loss at step 500: 25.939150
Minibatch accuracy: 82.0%
Validation accuracy: 77.0%
Minibatch loss at step 1000: 8.908557
Minibatch accuracy: 85.9%
Validation accuracy: 79.3%
Minibatch loss at step 1500: 3.650825
Minibatch accuracy: 88.3%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 0.541566
Minibatch accuracy: 95.3%
Validation accuracy: 82.5%
Minibatch loss at step 2500: 0.136494
Minibatch accuracy: 96.9%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 0.154153
Minibatch accuracy: 97.7%
Validation accuracy: 83.1%
Minibatch loss at step 3500: 1.769007
Minibatch accuracy: 96.9%
Validation accuracy: 83.2%
Minibatch loss at step 4000: 0.009514
Minibatch accuracy: 99.2%
Validation accuracy: 83.5%
Minibatch loss at step 4500: 4.677886
Minibatch accuracy: 93.8%
Validation accuracy: 80.5%
Test accuracy: 89.8%


---
Problem 2
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases = tf.Variable(tf.zeros([hidden_nodes]))    
  weights1 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases1 = tf.Variable(tf.zeros([num_labels]))   
    
  layer1_out = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  keep_prob = tf.placeholder(tf.float32)
  layer1_drop = tf.nn.dropout(layer1_out, keep_prob)

  # Training computation.
  logits = tf.matmul(layer1_drop, weights1) + biases1 + tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights1)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)


  valid_layer1_out = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_layer1_out, weights1) + biases1)
  test_layer1_out = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_layer1_out, weights1) + biases1)

In [12]:
num_steps = 5000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 485.180756
Minibatch accuracy: 8.6%
Validation accuracy: 14.0%
Minibatch loss at step 500: 37.755867
Minibatch accuracy: 73.4%
Validation accuracy: 78.0%
Minibatch loss at step 1000: 18.243610
Minibatch accuracy: 75.8%
Validation accuracy: 77.8%
Minibatch loss at step 1500: 8.530111
Minibatch accuracy: 71.9%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 4.183762
Minibatch accuracy: 82.0%
Validation accuracy: 81.1%
Minibatch loss at step 2500: 3.983388
Minibatch accuracy: 79.7%
Validation accuracy: 80.4%
Minibatch loss at step 3000: 6.976005
Minibatch accuracy: 75.8%
Validation accuracy: 81.5%
Minibatch loss at step 3500: 2.604526
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Minibatch loss at step 4000: 5.650119
Minibatch accuracy: 84.4%
Validation accuracy: 81.4%
Minibatch loss at step 4500: 11.288406
Minibatch accuracy: 79.7%
Validation accuracy: 79.9%
Test accuracy: 89.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [13]:
batch_size = 128
num_hidden_nodes1 = 1024
num_hidden_nodes2 = 256
num_hidden_nodes3 = 128
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)

  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal(
        [image_size * image_size, num_hidden_nodes1],
        stddev=np.sqrt(2.0 / (image_size * image_size)))
    )
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_nodes1)))
  biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
  weights3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_hidden_nodes3], stddev=np.sqrt(2.0 / num_hidden_nodes2)))
  biases3 = tf.Variable(tf.zeros([num_hidden_nodes3]))
  weights4 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes3, num_labels], stddev=np.sqrt(2.0 / num_hidden_nodes3)))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  lay2_train = tf.nn.relu(tf.matmul(lay1_train, weights2) + biases2)
  lay3_train = tf.nn.relu(tf.matmul(lay2_train, weights3) + biases3)
  logits = tf.matmul(lay3_train, weights4) + biases4
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 4000, 0.65, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  lay2_valid = tf.nn.relu(tf.matmul(lay1_valid, weights2) + biases2)
  lay3_valid = tf.nn.relu(tf.matmul(lay2_valid, weights3) + biases3)
  valid_prediction = tf.nn.softmax(tf.matmul(lay3_valid, weights4) + biases4)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  lay2_test = tf.nn.relu(tf.matmul(lay1_test, weights2) + biases2)
  lay3_test = tf.nn.relu(tf.matmul(lay2_test, weights3) + biases3)
  test_prediction = tf.nn.softmax(tf.matmul(lay3_test, weights4) + biases4)

In [14]:
num_steps = 18001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 2.382507
Minibatch accuracy: 10.9%
Validation accuracy: 10.1%
Minibatch loss at step 500: 0.377853
Minibatch accuracy: 86.7%
Validation accuracy: 84.9%
Minibatch loss at step 1000: 0.580725
Minibatch accuracy: 82.8%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 0.423568
Minibatch accuracy: 87.5%
Validation accuracy: 86.6%
Minibatch loss at step 2000: 0.096711
Minibatch accuracy: 97.7%
Validation accuracy: 86.4%
Minibatch loss at step 2500: 0.119986
Minibatch accuracy: 97.7%
Validation accuracy: 86.5%
Minibatch loss at step 3000: 0.160340
Minibatch accuracy: 96.1%
Validation accuracy: 86.3%
Minibatch loss at step 3500: 0.077874
Minibatch accuracy: 97.7%
Validation accuracy: 86.7%
Minibatch loss at step 4000: 0.059368
Minibatch accuracy: 98.4%
Validation accuracy: 87.0%
Minibatch loss at step 4500: 0.076814
Minibatch accuracy: 98.4%
Validation accuracy: 87.3%
Minibatc